# Entrenamiento Red Neuronal con Tensorflow

## Importar los paquetes necesarios para entrenamiento de la red neuronal

In [30]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

import os
import numpy as np
import skimage as sk
import skimage.io
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
#from online_training import *
import datetime

from torch.utils.tensorboard import SummaryWriter

Definir si se ejecutará el código en Google Colab o una PC con sistema operativo Windows.

En caso de ser Google Colab se establecerá la conexión con Google Drive, de lo contrario con el repositorio en GitHub.

In [31]:
is_googlecolab = False

## Definir ruta del repositorio con las imágenes para entrenar el modelo

In [32]:
if is_googlecolab:
    from google.colab import drive
    drive.mount('/content/drive')

    PATH        = '/content/drive/MyDrive/Colab Notebooks/DogFaceNet/data/dogfacenet/aligned/after_4_bis/' # Path to the directory of the saved dataset
    #PATH        = '/content/drive/MyDrive/Colab Notebooks/Fotos perros prueba/BaseDeDatos/'
    #PATH_SAVE   = '/content/drive/MyDrive/Colab Notebooks/DogFaceNet/output/history/'                      # Path to the directory where the history will be stored
    PATH_SAVE   = '/content/drive/MyDrive/GHT/DogFaceNet_shared/DogFaceNet_output/output/history/'
    #PATH_MODEL  = '/content/drive/MyDrive/Colab Notebooks/DogFaceNet/output/model/tensorflow_2021.03.30/'             # Path to the directory where the model will be stored
    PATH_MODEL  = '/content/drive/MyDrive/GHT/DogFaceNet_shared/DogFaceNet_output/output/model/tensorflow_2021.03.30.2/'
else:
    #!git clone https://gitlab.com/teamghtgcba/dogs-dataset-b-limpio.git
    #!git clone -b datasets_limpios_alineados https://gitlab.com/teamghtgcba/dogs-dataset-b-limpio.git

    PATH        = './dogs-dataset-b-limpio/data/' # Path to the directory of the saved dataset
    PATH_SAVE   = './dogs-dataset-b-limpio/output/history/'                      # Path to the directory where the history will be stored
    PATH_MODEL  = './dogs-dataset-b-limpio/output/model/tensorflow_2021.03.30/'             # Path to the directory where the model will be stored
    '''
    # Pruebas en Colab por performance al acceder a la base de datos de imágenes
    from google.colab import drive
    drive.mount('/content/drive')
    PATH_SAVE   = '/content/drive/MyDrive/GHT/DogFaceNet_shared/DogFaceNet_output/output/history/'
    PATH_MODEL  = '/content/drive/MyDrive/GHT/DogFaceNet_shared/DogFaceNet_output/output/model/tensorflow_2021.03.31/'
    '''

Validar que exista la ruta del dataset

In [33]:
assert os.path.isdir(PATH), '[Error] Provided PATH for dataset does not exist.'

### Importar offline_training

In [34]:
"""
DogFaceNet
Functions for offline training.
The online_training module should be prefered instead of this one.
offline_training will load all the dataset into computer memory.
Even if the training is slighty faster the computer can rapidly
run out of memory.

Licensed under the MIT License (see LICENSE for details)
Written by Guillaume Mougeot
"""

import tensorflow as tf
import numpy as np
from tqdm import tqdm_notebook


datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=8,
    zoom_range=0.1,
    fill_mode='nearest',
    channel_shift_range = 0.1
)

def single_apply_transform(image, datagen):
    """
    Apply a data preprocessing transformation to a single image
    Args:
        -image
        -ImageDataGenerator
    Return:
        -an image of the same shape of the input but transformed
    """
    image_exp = np.expand_dims(image,0)
    for x in datagen.flow(image_exp, batch_size=1):
        return x[0]

def apply_transform(images, datagen):
    """
    Apply a data preprocessing transformation to n images
    Args:
        -images
        -ImageDataGenerator
    Return:
        -images of the same shape of the inputs but transformed
    """
    for x in datagen.flow(images, batch_size=len(images), shuffle=False):
        return x

def define_triplets(images,labels,nbof_triplet = 10000 * 3, datagen=datagen):
    _,h,w,c = images.shape
    triplet_train = np.empty((nbof_triplet,h,w,c))
    y_triplet = np.empty(nbof_triplet)
    classes = np.unique(labels)
    for i in tqdm_notebook(range(0,nbof_triplet,3)):
        # Pick a class and chose two pictures from this class
        classAP = classes[np.random.randint(len(classes))]
        keep = np.equal(labels,classAP)
        keep_classAP = images[keep]
        keep_classAP_idx = labels[keep]
        idx_image1 = np.random.randint(len(keep_classAP))
        idx_image2 = np.random.randint(len(keep_classAP))
        while idx_image1 == idx_image2:
            idx_image2 = np.random.randint(len(keep_classAP))

        triplet_train[i] = single_apply_transform(keep_classAP[idx_image1],datagen)
        triplet_train[i+1] = single_apply_transform(keep_classAP[idx_image2],datagen)
        y_triplet[i] = keep_classAP_idx[idx_image1]
        y_triplet[i+1] = keep_classAP_idx[idx_image2]
        # Pick a class for the negative picture
        classN = classes[np.random.randint(len(classes))]
        while classN==classAP:
            classN = classes[np.random.randint(len(classes))]
        keep = np.equal(labels,classN)
        keep_classN = images[keep]
        keep_classN_idx = labels[keep]
        idx_image3 = np.random.randint(len(keep_classN))
        triplet_train[i+2] = single_apply_transform(keep_classN[idx_image3],datagen)
        y_triplet[i+2] = keep_classN_idx[idx_image3]
        
    return triplet_train, y_triplet


def shuffle_classes(images,labels):
    """
    Shuffles the classes
    """
    classes = np.unique(labels)
    np.random.shuffle(classes)
    
    shuffled_images = np.empty(images.shape)
    shuffled_labels = np.empty(labels.shape)
    idx = 0
    for i in range(len(classes)):
        keep_classes = np.equal(labels,classes[i])
        length = np.sum(keep_classes.astype(int))
        shuffled_labels[idx:idx+length] = labels[keep_classes]
        shuffled_images[idx:idx+length] = images[keep_classes]
        idx += length
    return shuffled_images, shuffled_labels

def global_define_hard_triplets(images,labels,predict,datagen=datagen):
    """
    Generates hard triplet for offline selection. It will consider the whole dataset.
    
    Args:
        -images: images from which the triplets will be created
        -labels: labels of the images
        -predict: predicted embeddings for the images by the trained model
        -alpha: threshold of the triplet loss
    Returns:
        -triplet
        -y_triplet: labels of the triplets
    """
    _,idx_classes = np.unique(labels,return_index=True)
    classes = labels[np.sort(idx_classes)]
    nbof_classes = len(classes)
    _,h,w,c = images.shape
    triplets = np.empty((3*len(predict),h,w,c))
    y_triplets = np.empty(3*len(predict))
    
    idx_triplets = 0
    idx_images = 0
    
    for i in range(nbof_classes):
        keep_class = np.equal(labels,classes[i])
        
        #predict_class = mask_class.dot(predict)
        predict_other = np.copy(predict)
        for j in range(len(predict)):
            if keep_class[j]:
                predict_other[j] += np.inf
        
        keep_predict_class = predict[keep_class]
        
        for j in range(len(keep_predict_class)):
            # Computes the distance between the current vector and the vectors in the class
            dist_class = np.sum(np.square(keep_predict_class-keep_predict_class[j]),axis=-1)
            
            # Add the anchor
            triplets[idx_triplets] = single_apply_transform(images[idx_images+j],datagen)
            y_triplets[idx_triplets] = labels[idx_images+j]
            
            # Add the hard positive
            triplets[idx_triplets+1] = single_apply_transform(images[idx_images+np.argmax(dist_class)],datagen)
            y_triplets[idx_triplets+1] = labels[idx_images+np.argmax(dist_class)]
            
            # Computes the distance between the current vector and the vectors of the others classes
            dist_other = np.sum(np.square(predict_other-keep_predict_class[j]),axis=-1)
            
            # Add the hard negative
            triplets[idx_triplets+2] = single_apply_transform(images[np.argmin(dist_other)],datagen)
            y_triplets[idx_triplets+2] = labels[np.argmin(dist_other)]
            
            idx_triplets += 3
        
        idx_images += len(keep_predict_class)
        
    return triplets, y_triplets


def define_hard_triplets(images,labels,predict,class_subset_size=10, add=100*3):
    """
    Generates hard triplet for offline selection
    
    Args:
        -images: images from which the triplets will be created
        -labels: labels of the images
        -predict: predicted embeddings for the images by the trained model
        -alpha: threshold of the triplet loss
        -class_subset_class: number of classes in a subset
        -
    Returns:
        -triplet
        -y_triplet: labels of the triplets
    """
    _,idx_classes = np.unique(labels,return_index=True)
    classes = labels[np.sort(idx_classes)]
    nbof_classes = len(classes)
    _,h,w,c = images.shape
    triplets = np.empty((3*len(predict)+add*(nbof_classes//class_subset_size + 1),h,w,c))
    y_triplets = np.empty(3*len(predict)+add*(nbof_classes//class_subset_size + 1))
    idx = 0
    for i in tqdm_notebook(range(0,len(classes),class_subset_size)):
        selected_classes = classes[i:i+class_subset_size]
        keep_classes = np.array([labels[j] in selected_classes for j in range(len(labels))])
        
        selected_predict = predict[keep_classes]
        length = len(selected_predict)*3
        
        triplets_tmp,y_triplets_tmp = global_define_hard_triplets(
                                                                images[keep_classes],
                                                                labels[keep_classes],
                                                                selected_predict
                                                            )
        print(len(triplets_tmp))
        
        triplets[idx:idx+length] = triplets_tmp
        y_triplets[idx:idx+length] = y_triplets_tmp
        
        triplets[idx+length:idx+length+add], y_triplets[idx+length:idx+length+add] = define_triplets(images, labels, add)
        
        idx += len(triplets_tmp) + add
    return triplets, y_triplets

### Importar online_training

In [35]:
"""
DogFaceNet
Functions for training on bigger datasets then offline_training module.
It does not load all the dataset into memory but just a part of it.
It mainly relies on keras data generators.
It contains:
 - Offline triplet generator: for soft and hard triplets
 - Online triplet generator: for soft and hard triplets

Licensed under the MIT License (see LICENSE for details)
Written by Guillaume Mougeot
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

import os
import pickle
import numpy as np
import skimage as sk
import skimage.io
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
#from offline_training import *
from math import isnan

SIZE = (224,224,3)

def define_triplets_batch(filenames,labels,nbof_triplet = 21 * 3):
    """
    Generates offline soft triplet.
    Given a list of file names of pictures, their specific label and
    a number of triplet images, returns an array of triplet of images
    and their specific labels.

    Args:
     - filenames: array of strings. List of file names of the pictures. 
     - labels: array of integers.
     - nbof_triplet: integer. Has to be a multiple of 3.
     
     Returns:
     - triplet_train: array of pictures --> a 4D array. 
     - y_triplet: array of integers of same dimension as the first
     dimension of triplet_train. Contains the labels of the pictures.
    """
    triplet_train = []
    y_triplet = np.empty(nbof_triplet)
    classes = np.unique(labels)
    for i in range(0,nbof_triplet,3):
        # Pick a class and chose two pictures from this class
        classAP = classes[np.random.randint(len(classes))]
        keep = np.equal(labels,classAP)
        keep_classAP = filenames[keep]
        keep_classAP_idx = labels[keep]
        idx_image1 = np.random.randint(len(keep_classAP))
        idx_image2 = np.random.randint(len(keep_classAP))
        while idx_image1 == idx_image2:
            idx_image2 = np.random.randint(len(keep_classAP))

        triplet_train += [keep_classAP[idx_image1]]
        triplet_train += [keep_classAP[idx_image2]]
        y_triplet[i] = keep_classAP_idx[idx_image1]
        y_triplet[i+1] = keep_classAP_idx[idx_image2]
        # Pick a class for the negative picture
        classN = classes[np.random.randint(len(classes))]
        while classN==classAP:
            classN = classes[np.random.randint(len(classes))]
        keep = np.equal(labels,classN)
        keep_classN = filenames[keep]
        keep_classN_idx = labels[keep]
        idx_image3 = np.random.randint(len(keep_classN))
        triplet_train += [keep_classN[idx_image3]]
        y_triplet[i+2] = keep_classN_idx[idx_image3]
        
    return triplet_train, y_triplet

def define_hard_triplets_batch(filenames,labels,predict,nbof_triplet=21*3, use_neg=True, use_pos=True):
    """
    [DEPRECATED] Use define_adaptive_hard_triplets_batch instead!
    Generates hard triplet for offline selection. It will consider the whole dataset.
    
    Args:
        -images: images from which the triplets will be created
        -labels: labels of the images
        -predict: predicted embeddings for the images by the trained model
        -alpha: threshold of the triplet loss
    Returns:
        -triplet
        -y_triplet: labels of the triplets
    """
    # Check if we have the right number of triplets
    assert nbof_triplet%3 == 0
    
    _,idx_classes = np.unique(labels,return_index=True)
    classes = labels[np.sort(idx_classes)]
    
    triplets = []
    y_triplets = np.empty(nbof_triplet)
    
    for i in range(0,nbof_triplet,3):
        # Chooses the first class randomly
        keep = np.equal(labels,classes[np.random.randint(len(classes))])
        keep_filenames = filenames[keep]
        keep_labels = labels[keep]
        
        # Chooses the first image among this class randomly
        idx_image1 = np.random.randint(len(keep_labels))
        
        
        # Computes the distance between the chosen image and the rest of the class
        if use_pos:
            dist_class = np.sum(np.square(predict[keep]-predict[keep][idx_image1]),axis=-1)

            idx_image2 = np.argmax(dist_class)
        else:
            idx_image2 = np.random.randint(len(keep_labels))
            i = 0
            while idx_image1==idx_image2:
                idx_image2 = np.random.randint(len(keep_labels))
                # Just to prevent endless loop:
                i += 1
                if i == 1000:
                    print("[Error: define_hard_triplets_batch] Endless loop.")
                    break
        
        triplets += [keep_filenames[idx_image1]]
        y_triplets[i] = keep_labels[idx_image1]
        triplets += [keep_filenames[idx_image2]]
        y_triplets[i+1] = keep_labels[idx_image2]
        
        
        # Computes the distance between the chosen image and the rest of the other classes
        not_keep = np.logical_not(keep)
        
        if use_neg:
            dist_other = np.sum(np.square(predict[not_keep]-predict[keep][idx_image1]),axis=-1)
            idx_image3 = np.argmin(dist_other) 
        else:
            idx_image3 = np.random.randint(len(filenames[not_keep]))
            
        triplets += [filenames[not_keep][idx_image3]]
        y_triplets[i+2] = labels[not_keep][idx_image3]

    #return triplets, y_triplets
    return np.array(triplets), y_triplets

def define_adaptive_hard_triplets_batch(filenames,labels,predict,nbof_triplet=21*3, use_neg=True, use_pos=True):
    """
    Generates hard triplet for offline selection. It will consider the whole dataset.
    This function will also return the predicted values.
    
    Args:
        -images: images from which the triplets will be created
        -labels: labels of the images
        -predict: predicted embeddings for the images by the trained model
        -alpha: threshold of the triplet loss
    Returns:
        -triplets
        -y_triplets: labels of the triplets
        -pred_triplets: predicted embeddings of the triplets
    """
    # Check if we have the right number of triplets
    assert nbof_triplet%3 == 0
    
    _,idx_classes = np.unique(labels,return_index=True)
    classes = labels[np.sort(idx_classes)]
    
    triplets = []
    y_triplets = np.empty(nbof_triplet)
    pred_triplets = np.empty((nbof_triplet,predict.shape[-1]))
    
    for i in range(0,nbof_triplet,3):
        # Chooses the first class randomly
        keep = np.equal(labels,classes[np.random.randint(len(classes))])
        keep_filenames = filenames[keep]
        keep_labels = labels[keep]
        
        # Chooses the first image among this class randomly
        idx_image1 = np.random.randint(len(keep_labels))
        
        
        # Computes the distance between the chosen image and the rest of the class
        if use_pos:
            dist_class = np.sum(np.square(predict[keep]-predict[keep][idx_image1]),axis=-1)

            idx_image2 = np.argmax(dist_class)
        else:
            idx_image2 = np.random.randint(len(keep_labels))
            j = 0
            while idx_image1==idx_image2:
                idx_image2 = np.random.randint(len(keep_labels))
                # Just to prevent endless loop:
                j += 1
                if j == 1000:
                    print("[Error: define_hard_triplets_batch] Endless loop.")
                    break
        
        triplets += [keep_filenames[idx_image1]]
        y_triplets[i] = keep_labels[idx_image1]
        pred_triplets[i] = predict[keep][idx_image1]
        triplets += [keep_filenames[idx_image2]]
        y_triplets[i+1] = keep_labels[idx_image2]
        pred_triplets[i+1] = predict[keep][idx_image2]
        
        # Computes the distance between the chosen image and the rest of the other classes
        not_keep = np.logical_not(keep)
        
        if use_neg:
            dist_other = np.sum(np.square(predict[not_keep]-predict[keep][idx_image1]),axis=-1)
            idx_image3 = np.argmin(dist_other) 
        else:
            idx_image3 = np.random.randint(len(filenames[not_keep]))
            
        triplets += [filenames[not_keep][idx_image3]]
        y_triplets[i+2] = labels[not_keep][idx_image3]
        pred_triplets[i+2] = predict[not_keep][idx_image3]

    return np.array(triplets), y_triplets, pred_triplets

def load_images(filenames):
    """
    Use scikit-image library to load the pictures from files to numpy array.
    """
    h,w,c = SIZE
    images = np.empty((len(filenames),h,w,c))
    for i,f in enumerate(filenames):
        images[i] = sk.io.imread(f)/255.0
    return images

def image_generator(filenames, labels, batch_size=63, use_aug=True, datagen=datagen):
    """
    Training generator for soft triplets.
    """
    while True:
        f_triplet, y_triplet = define_triplets_batch(filenames, labels, batch_size)
        i_triplet = load_images(f_triplet)
        if use_aug:
            i_triplet = apply_transform(i_triplet, datagen)
        yield (i_triplet, y_triplet)

def hard_image_generator(filenames, labels, predict, batch_size=63, use_neg=True, use_pos=True, use_aug=True, datagen=datagen):
    """
    Training generator for offline hard triplets.
    """
    while True:
        f_triplet, y_triplet = define_hard_triplets_batch(filenames, labels, predict, batch_size, use_neg=use_neg, use_pos=use_pos)
        i_triplet = load_images(f_triplet)
        if use_aug:
            i_triplet = apply_transform(i_triplet, datagen)
        yield (i_triplet, y_triplet)

def predict_generator(filenames, batch_size=32):
    """
    Prediction generator.
    """
    for i in range(0,len(filenames),batch_size):
        images_batch = load_images(filenames[i:i+batch_size])
        yield images_batch

def online_hard_image_generator(
    filenames,
    labels,
    model,
    batch_size=63,
    nbof_subclasses=10,
    use_neg=True,
    use_pos=True,
    use_aug=True,
    datagen=datagen):
    """
    Generator to select online hard triplets for training.
    
    Arguments:
        -filenames
        -labels
    """
    while True:
        # Select a certain amount of subclasses
        classes = np.unique(labels)
        subclasses = np.random.choice(classes,size=nbof_subclasses,replace=False)
        
        keep_classes = np.equal(labels,subclasses[0])
        for i in range(1,len(subclasses)):
            keep_classes = np.logical_or(keep_classes,np.equal(labels,subclasses[i]))
        subfilenames = filenames[keep_classes]
        sublabels = labels[keep_classes]
        predict = model.predict_generator(predict_generator(subfilenames, 32),
                                          steps=np.ceil(len(subfilenames)/32))
        
        f_triplet, y_triplet = define_hard_triplets_batch(subfilenames, sublabels, predict, batch_size, use_neg=use_neg, use_pos=use_pos)
        i_triplet = load_images(f_triplet)
        if use_aug:
            i_triplet = apply_transform(i_triplet, datagen)
        yield (i_triplet, y_triplet)

def online_adaptive_hard_image_generator(
    filenames,                  # Absolute path of the images
    labels,                     # Labels of the images
    model,                      # A keras model
    loss,                       # Current loss of the model
    batch_size      =63,        # Batch size (has to be a multiple of 3 for dogfacenet)
    nbof_subclasses =10,        # Number of subclasses from which the triplets will be selected
    use_aug         =True,      # Use data augmentation?
    datagen         =datagen):  # Data augmentation parameter
    """
    Generator to select online hard triplets for training.
    Include an adaptive control on the number of hard triplets included during the training.
    """
    
    hard_triplet_ratio = 0
    nbof_hard_triplets = 0
    while True:
        # Select a certain amount of subclasses
        classes = np.unique(labels)
        # In order to limit the number of computation for prediction,
        # we will not computes nbof_subclasses predictions for the hard triplets generation,
        # but int(nbof_subclasses*hard_triplet_ratio)+2, which means that the higher the
        # accuracy is the more prediction are going to be computed.
        subclasses = np.random.choice(classes,size=int(nbof_subclasses*hard_triplet_ratio)+2,replace=False)
        
        keep_classes = np.equal(labels,subclasses[0])
        for i in range(1,len(subclasses)):
            keep_classes = np.logical_or(keep_classes,np.equal(labels,subclasses[i]))
        subfilenames = filenames[keep_classes]
        sublabels = labels[keep_classes]
        predict = model.predict_generator(predict_generator(subfilenames, 32),
                                          steps=int(np.ceil(len(subfilenames)/32)))
        
        
        
        
        
        f_triplet_hard, y_triplet_hard, predict_hard = define_adaptive_hard_triplets_batch(subfilenames, sublabels, predict, nbof_hard_triplets*3, use_neg=True, use_pos=True)
        f_triplet_soft, y_triplet_soft, predict_soft = define_adaptive_hard_triplets_batch(subfilenames, sublabels, predict, batch_size-nbof_hard_triplets*3, use_neg=False, use_pos=False)

        f_triplet = np.append(f_triplet_hard,f_triplet_soft)
        y_triplet = np.append(y_triplet_hard,y_triplet_soft)

        predict = np.append(predict_hard, predict_soft, axis=0)
        
        # Proportion of hard triplets in the generated batch
        #hard_triplet_ratio = max(0,1.2/(1+np.exp(-10*acc+5.3))-0.19)
        hard_triplet_ratio = np.exp(-loss * 10 / batch_size)

        if isnan(hard_triplet_ratio):
            hard_triplet_ratio = 0
        nbof_hard_triplets = int(batch_size//3 * hard_triplet_ratio)
        
        i_triplet = load_images(f_triplet)
        if use_aug:
            i_triplet = apply_transform(i_triplet, datagen)
            
        # Potential modif for different losses: re-labels the dataset from 0 to nbof_subclasses
        # dict_subclass = {subclasses[i]:i for i in range(nbof_subclasses)}
        # ridx_y_triplet = [dict_subclass[y_triplet[i]] for i in range(len(y_triplet))]
        
        yield (i_triplet, y_triplet)


## Definir los valores de entrenamiento de la red neuronal

In [36]:
SIZE        = (224,224,3)                               # Size of the input images
TEST_SPLIT  = 0.05                                       # Test ratio
VAL_SPLIT  = 0.05                                       # Val ratio

NET_NAME    = 'tensorflow.dogfacenet_tf'                   # Network saved name
START_EPOCH = 0                                         # Start the training at a specified epoch
NBOF_EPOCHS = 250                                       # Number        
STEPS_PER_EPOCH = 300                                   # Number of steps per epoch
VALIDATION_STEPS = 30    

### Mejora de Luciano para precargar las imágenes

In [37]:
import time
def online_adaptive_hard_image_generator_pre(*args, **kwargs):
  start = time.time()
  out = []
  for x in online_adaptive_hard_image_generator(*args, **kwargs):
    out.append(x)
    print("Loading data for step #{}/{}".format(len(out), STEPS_PER_EPOCH))
    if len(out) == STEPS_PER_EPOCH:
      break
  end = time.time()
  print("Epoch data loaded ({} seconds)".format(end - start))
  for x in out:
    yield x

## Entrenamiento de la red neuronal

### Cargar dataset de imágenes de perros

In [38]:
print('Loading the dataset...')

import os, random
carpetas = sorted([o for o in os.listdir(PATH) if os.path.isdir(os.path.join(PATH, o))])
random.seed(4)
random.shuffle(carpetas)
print(carpetas[:10])

carpetas2 = [(PATH + c, None, os.listdir(os.path.join(PATH, c))) for c in carpetas]

filenames = np.empty(0)
labels = np.empty(0)
idx = 0
for root, dirs, files in carpetas2:
    # Directorio / / archivos por directorio (list())
    #print('{} {} {}'.format(root,dirs,files))
    if len(files)>1:
        for i in range(len(files)):
            files[i] = root + '/' + files[i]
        filenames = np.append(filenames,files)
        # Recupera la ruta de todos los archivos
        #print('{}'.format(filenames))
        labels = np.append(labels,np.ones(len(files))*idx)
        # Crea una lista de etiquetas "nombre de las carpetas". Posteriormente se obtendrá las etiquetas únicas
        #print('{}'.format(labels))
        idx += 1

print(filenames)
print(labels)

Loading the dataset...
['50430193', '51293014', '51070822', '51236085', '51298788', '1301', '51286528', '50894180', '51217312', '51202526']
['./dogs-dataset-b-limpio/data/50430193/2.jpg'
 './dogs-dataset-b-limpio/data/50430193/4.jpg'
 './dogs-dataset-b-limpio/data/50430193/5.jpg' ...
 './dogs-dataset-b-limpio/data/50556157/3.jpg'
 './dogs-dataset-b-limpio/data/50138359/1.jpg'
 './dogs-dataset-b-limpio/data/50138359/0.jpg']
[    0.     0.     0. ... 12475. 12476. 12476.]


In [39]:
'''
print('Loading the dataset...')

filenames = np.empty(0)
labels = np.empty(0)
idx = 0
for root, dirs, files in os.walk(PATH):
    # Directorio / / archivos por directorio (list())
    #print('{} {} {}'.format(root,dirs,files))
    if len(files)>1:
        for i in range(len(files)):
            files[i] = root + '/' + files[i]
        filenames = np.append(filenames,files)
        # Recupera la ruta de todos los archivos
        #print('{}'.format(filenames))
        labels = np.append(labels,np.ones(len(files))*idx)
        # Crea una lista de etiquetas "nombre de las carpetas". Posteriormente se obtendrá las etiquetas únicas
        #print('{}'.format(labels))
        idx += 1
assert len(labels)!=0, '[Error] No data provided.'
print('Done.')

print(filenames)
print(labels)
'''

'\nprint(\'Loading the dataset...\')\n\nfilenames = np.empty(0)\nlabels = np.empty(0)\nidx = 0\nfor root, dirs, files in os.walk(PATH):\n    # Directorio / / archivos por directorio (list())\n    #print(\'{} {} {}\'.format(root,dirs,files))\n    if len(files)>1:\n        for i in range(len(files)):\n            files[i] = root + \'/\' + files[i]\n        filenames = np.append(filenames,files)\n        # Recupera la ruta de todos los archivos\n        #print(\'{}\'.format(filenames))\n        labels = np.append(labels,np.ones(len(files))*idx)\n        # Crea una lista de etiquetas "nombre de las carpetas". Posteriormente se obtendrá las etiquetas únicas\n        #print(\'{}\'.format(labels))\n        idx += 1\nassert len(labels)!=0, \'[Error] No data provided.\'\nprint(\'Done.\')\n\nprint(filenames)\nprint(labels)\n'

### Separar dataset para entrenamiento y test

In [40]:
print('Total number of imported pictures: {:d}'.format(len(labels)))
nbof_classes = len(np.unique(labels))
print('Total number of classes: {:d}'.format(nbof_classes))

# Split the dataset.
print('\nSplit por las clases de perros que se tienen, no por las imágenes')

nbof_test = int(TEST_SPLIT*nbof_classes)
nbof_val = int(VAL_SPLIT*nbof_classes)

keep_test = np.less(labels, nbof_test) # 111 0000000000000
keep_val_1 = np.greater_equal(labels, nbof_test) # 000 1111111111111
keep_val_2 = np.less(labels, nbof_test + nbof_val) # 1111111111111 000000
keep_val = np.logical_and(keep_val_1, keep_val_2) # 000 1111111111 000000
keep_train = np.greater_equal(labels, nbof_test + nbof_val) # 000000000 1111
'''
print(keep_test)
print(keep_val)
print(keep_train)
print(np.sum(keep_test), np.sum(keep_val), np.sum(keep_train))
'''

filenames_test = filenames[keep_test]
labels_test = labels[keep_test]

filenames_train = filenames[keep_train]
labels_train = labels[keep_train]

filenames_val = filenames[keep_val]
labels_val = labels[keep_val]

print("Number of training data: {} pictures".format(str(len(filenames_train))))
print("Number of training classes: {} ({} %)".format(str(nbof_classes-nbof_test-nbof_val), 1 - TEST_SPLIT - VAL_SPLIT))
print("Number of testing data: {} pictures".format(str(len(filenames_test))))
print("Number of testing classes: {} ({} %)".format(str(nbof_test), TEST_SPLIT))
print("Number of validation data: {} pictures".format(str(len(filenames_val))))
print("Number of validation classes: {} ({} %)".format(str(nbof_val), VAL_SPLIT))

Total number of imported pictures: 41142
Total number of classes: 12477

Split por las clases de perros que se tienen, no por las imágenes
Number of training data: 37110 pictures
Number of training classes: 11231 (0.8999999999999999 %)
Number of testing data: 2025 pictures
Number of testing classes: 623 (0.05 %)
Number of validation data: 2007 pictures
Number of validation classes: 623 (0.05 %)


## Configuración de la red (declarar funciones)

In [41]:
alpha = 0.3
def triplet(y_true,y_pred):
    
    a = y_pred[0::3]
    p = y_pred[1::3]
    n = y_pred[2::3]
    
    ap = K.sum(K.square(a-p),-1)
    an = K.sum(K.square(a-n),-1)

    return K.sum(tf.nn.relu(ap - an + alpha))

def triplet_acc(y_true,y_pred):
    a = y_pred[0::3]
    p = y_pred[1::3]
    n = y_pred[2::3]
    
    ap = K.sum(K.square(a-p),-1)
    an = K.sum(K.square(a-n),-1)
    
    return K.less(ap+alpha,an)

## Implementar red neuronal

In [42]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Add, GlobalAveragePooling2D, DepthwiseConv2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

"""
Model number 12: Paper version: a modified ResNet with Dropout layers and without bottleneck layers
"""

print('Defining model {:s} ...'.format(NET_NAME))

emb_size = 32

inputs = Input(shape=SIZE)

x = Conv2D(16, (7, 7), (2, 2), use_bias=False, activation='relu', padding='same')(inputs)
x = BatchNormalization()(x)
x = MaxPooling2D((3,3))(x)

for layer in [16,32,64,128,512]:

    x = Conv2D(layer, (3, 3), strides=(2,2), use_bias=False, activation='relu', padding='same')(x)
    r = BatchNormalization()(x)
    
    x = Conv2D(layer, (3, 3), use_bias=False, activation='relu', padding='same')(r)
    x = BatchNormalization()(x)
    r = Add()([r,x])
    
    x = Conv2D(layer, (3, 3), use_bias=False, activation='relu', padding='same')(r)
    x = BatchNormalization()(x)
    x = Add()([r,x])
    

x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(emb_size, use_bias=False)(x)
outputs = Lambda(lambda x: tf.nn.l2_normalize(x,axis=-1))(x)

model = tf.keras.Model(inputs,outputs)

model.compile(loss=triplet,
            optimizer='adam',
            metrics=[triplet_acc])

print('Done.')

Defining model tensorflow.dogfacenet_tf ...
Done.


In [43]:
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 112, 112, 16) 2352        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_16 (BatchNo (None, 112, 112, 16) 64          conv2d_16[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 37, 37, 16)   0           batch_normalization_16[0][0]     
____________________________________________________________________________________________

## Si estamos reanudando entrenamiento, recargamos el modelo desde el checkpoint

In [44]:
reanudar_entrenamiento = True

if reanudar_entrenamiento:
    RESUME_TRAINING_PATH_MODEL = "./dogs-dataset-b-limpio/output_a/model/tensorflow_2021.03.30/"
    load_epoch = 249
    old_model = tf.keras.models.load_model(
        '{:s}{:s}.{:d}.pb'.format(RESUME_TRAINING_PATH_MODEL,NET_NAME,load_epoch), 
        custom_objects={'triplet':triplet,'triplet_acc':triplet_acc})
    model.set_weights(old_model.get_weights())
    START_EPOCH = 250
    NBOF_EPOCHS = 100000

## Declarar Tensorboard

Declarar directorio donde se guardarán los resultados por cada época en la red neuronal.

In [45]:
# Path for saving the logs
path = 'logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=path, histogram_freq=1)

## Entrenar red neuronal

In [ ]:
from datetime import datetime

histories = []
crt_loss = 0.6
crt_acc = 0
batch_size = 3*10
nbof_subclasses = 40

if reanudar_entrenamiento:
    loss_resume,val_loss_resume,acc_resume,val_acc_resume = np.load('{:s}{:s}.{:d}.npy'.format(PATH_SAVE.replace("output", "output_a"),NET_NAME,load_epoch))
    crt_loss = loss_resume[-1]
    crt_acc = acc_resume[-1]

# Create saving folders
if not os.path.isdir(PATH_MODEL):
    os.makedirs(PATH_MODEL)
if not os.path.isdir(PATH_SAVE):
    os.makedirs(PATH_SAVE)

# Bug fixed: keras models are to be initialized by a training on a single batch
for images_batch,labels_batch in online_adaptive_hard_image_generator(
    filenames_train,
    labels_train,
    model,
    crt_acc,
    batch_size,
    nbof_subclasses=nbof_subclasses):
    h = model.train_on_batch(images_batch,labels_batch)
    break

writer = SummaryWriter('{:s}{:s}'.format(PATH_SAVE,NET_NAME))

for i in range(START_EPOCH,START_EPOCH+NBOF_EPOCHS):
    print("Beginning epoch number: "+str(i))

    hard_triplet_ratio = np.exp(-crt_loss * 10 / batch_size)
    nbof_hard_triplets = int(batch_size//3 * hard_triplet_ratio)
    
    print("Current hard triplet ratio: " + str(hard_triplet_ratio))
    
    histories += [model.fit_generator(
        online_adaptive_hard_image_generator_pre(filenames_train,labels_train,model,crt_loss,batch_size,nbof_subclasses=nbof_subclasses),
        #online_adaptive_hard_image_generator(filenames_train,labels_train,model,crt_loss,batch_size,nbof_subclasses=nbof_subclasses),
        steps_per_epoch=STEPS_PER_EPOCH,
        epochs=1,
        validation_data=image_generator(filenames_val,labels_val,batch_size,use_aug=False),
        validation_steps=VALIDATION_STEPS,
        callbacks = [tensorboard_callback]
        )]
    
    crt_loss = histories[-1].history['loss'][0]
    crt_acc = histories[-1].history['triplet_acc'][0]

    crt_loss_val = histories[-1].history['val_loss'][0]
    crt_acc_val = histories[-1].history['val_triplet_acc'][0]
    
    with open('salida.txt', 'a') as f:
        f.write(f'{str(datetime.now())}\tepoch: {i}/{NBOF_EPOCHS},\tloss: {crt_loss},\ttriplet_acc: {crt_acc},\tval_loss: {crt_loss_val},\tval_triplet_acc: {crt_acc_val}\n')

    model.save('{:s}{:s}.{:d}.pb'.format(PATH_MODEL,NET_NAME,i))
    
    # Save history
    loss = np.empty(0)
    val_loss = np.empty(0)
    acc = np.empty(0)
    val_acc = np.empty(0)

    for history in histories:
        loss = np.append(loss,history.history['loss'])
        val_loss = np.append(val_loss,history.history['val_loss'])
        acc = np.append(acc,history.history['triplet_acc'])
        val_acc = np.append(val_acc,history.history['val_triplet_acc'])

    history_ = np.array([loss,val_loss,acc,val_acc])
    np.save('{:s}{:s}.{:d}.npy'.format(PATH_SAVE,NET_NAME,i),history_)
    
    writer.add_scalar("loss", crt_loss, i)
    writer.add_scalar("val_loss", crt_loss_val, i)
    writer.add_scalar("triplet_acc", crt_acc, i)
    writer.add_scalar("val_triplet_acc", crt_acc_val, i)

writer.flush()
writer.close()

Beginning epoch number: 250
Current hard triplet ratio: 0.6301147424954741


/anaconda/envs/py38_tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Loading data for step #1/300
Loading data for step #2/300
Loading data for step #3/300
Loading data for step #4/300
Loading data for step #5/300
Loading data for step #6/300
Loading data for step #7/300
Loading data for step #8/300
Loading data for step #9/300
Loading data for step #10/300
Loading data for step #11/300
Loading data for step #12/300
Loading data for step #13/300
Loading data for step #14/300
Loading data for step #15/300
Loading data for step #16/300
Loading data for step #17/300
Loading data for step #18/300
Loading data for step #19/300
Loading data for step #20/300
Loading data for step #21/300
Loading data for step #22/300
Loading data for step #23/300
Loading data for step #24/300
Loading data for step #25/300
Loading data for step #26/300
Loading data for step #27/300
Loading data for step #28/300
Loading data for step #29/300
Loading data for step #30/300
Loading data for step #31/300
Loading data for step #32/300
Loading data for step #33/300
Loading data for st

Loading data for step #269/300
Loading data for step #270/300
Loading data for step #271/300
Loading data for step #272/300
Loading data for step #273/300
Loading data for step #274/300
Loading data for step #275/300
Loading data for step #276/300
Loading data for step #277/300
Loading data for step #278/300
Loading data for step #279/300
Loading data for step #280/300
Loading data for step #281/300
Loading data for step #282/300
Loading data for step #283/300
Loading data for step #284/300
Loading data for step #285/300
Loading data for step #286/300
Loading data for step #287/300
Loading data for step #288/300
Loading data for step #289/300
Loading data for step #290/300
Loading data for step #291/300
Loading data for step #292/300
Loading data for step #293/300
Loading data for step #294/300
Loading data for step #295/300
Loading data for step #296/300
Loading data for step #297/300
Loading data for step #298/300
Loading data for step #299/300
Loading data for step #300/300
Epoch da

/anaconda/envs/py38_tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/anaconda/envs/py38_tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


...

/anaconda/envs/py38_tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/anaconda/envs/py38_tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Loading data for step #1/300
Loading data for step #2/300
Loading data for step #3/300
Loading data for step #4/300
Loading data for step #5/300
Loading data for step #6/300
Loading data for step #7/300
Loading data for step #8/300
Loading data for step #9/300
Loading data for step #10/300
Loading data for step #11/300
Loading data for step #12/300
Loading data for step #13/300
Loading data for step #14/300
Loading data for step #15/300
Loading data for step #16/300
Loading data for step #17/300
Loading data for step #18/300
Loading data for step #19/300
Loading data for step #20/300
Loading data for step #21/300
Loading data for step #22/300
Loading data for step #23/300
Loading data for step #24/300
Loading data for step #25/300
Loading data for step #26/300
Loading data for step #27/300
Loading data for step #28/300
Loading data for step #29/300
Loading data for step #30/300
Loading data for step #31/300
Loading data for step #32/300
Loading data for step #33/300
Loading data for st

Loading data for step #269/300
Loading data for step #270/300
Loading data for step #271/300
Loading data for step #272/300
Loading data for step #273/300
Loading data for step #274/300
Loading data for step #275/300
Loading data for step #276/300
Loading data for step #277/300
Loading data for step #278/300
Loading data for step #279/300
Loading data for step #280/300
Loading data for step #281/300
Loading data for step #282/300
Loading data for step #283/300
Loading data for step #284/300
Loading data for step #285/300
Loading data for step #286/300
Loading data for step #287/300
Loading data for step #288/300
Loading data for step #289/300
Loading data for step #290/300
Loading data for step #291/300
Loading data for step #292/300
Loading data for step #293/300
Loading data for step #294/300
Loading data for step #295/300
Loading data for step #296/300
Loading data for step #297/300
Loading data for step #298/300
Loading data for step #299/300
Loading data for step #300/300
Epoch da

/anaconda/envs/py38_tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/anaconda/envs/py38_tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Loading data for step #1/300
Loading data for step #2/300
Loading data for step #3/300
Loading data for step #4/300
Loading data for step #5/300
Loading data for step #6/300
Loading data for step #7/300
Loading data for step #8/300
Loading data for step #9/300
Loading data for step #10/300
Loading data for step #11/300
Loading data for step #12/300
Loading data for step #13/300
Loading data for step #14/300
Loading data for step #15/300
Loading data for step #16/300
Loading data for step #17/300
Loading data for step #18/300
Loading data for step #19/300
Loading data for step #20/300
Loading data for step #21/300
Loading data for step #22/300
Loading data for step #23/300
Loading data for step #24/300
Loading data for step #25/300
Loading data for step #26/300
Loading data for step #27/300
Loading data for step #28/300
Loading data for step #29/300


## Mostrar gráfico con el loss y accuracy de la red neuronal

La red neuronal guarda un archivo .npy por cada época. Los valores que almacena son: 'loss', 'val_loss', 'triplet_acc', 'val_triplet_acc'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
PATH_SAVE_LOG = '{0}{1}{2}{3}{4}'.format(PATH_SAVE, NET_NAME, '.', (NBOF_EPOCHS - 1), '.npy')

In [ ]:
def mostrar_grafico_de_red_neuronal_loss(PATH_SAVE_LOG):
    log_model_train = np.load(PATH_SAVE_LOG)
    df_log_model_train = pd.DataFrame(log_model_train)
    df_log_model_train.index = ['loss', 'val_loss', 'triplet_acc', 'val_triplet_acc']
    
    df_log_model_train.T[df_log_model_train.index[0]].plot(legend=True)
    df_log_model_train.T[df_log_model_train.index[1]].plot(legend=True)

In [ ]:
mostrar_grafico_de_red_neuronal_loss(PATH_SAVE_LOG)

In [ ]:
def mostrar_grafico_de_red_neuronal_acc(PATH_SAVE_LOG):
    log_model_train = np.load(PATH_SAVE_LOG)
    df_log_model_train = pd.DataFrame(log_model_train)
    df_log_model_train.index = ['loss', 'val_loss', 'triplet_acc', 'val_triplet_acc']
    
    df_log_model_train.T[df_log_model_train.index[2]].plot(legend=True)
    df_log_model_train.T[df_log_model_train.index[3]].plot(legend=True)

In [ ]:
mostrar_grafico_de_red_neuronal_acc(PATH_SAVE_LOG)

In [ ]:
log_model_train = np.load(PATH_SAVE_LOG)
log_model_train

In [ ]:
NET_NAME_ = PATH_SAVE + 'tensorflow.dogfacenet_tf.81.npy'

mostrar_grafico_de_red_neuronal_loss(NET_NAME_)

In [ ]:
mostrar_grafico_de_red_neuronal_acc(NET_NAME_)

## Visualizar resultado de Tensorboard

In [ ]:
%tensorflow_version 2.x

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir 'logs/fit/20210330-230023'

# Despliegue en Flask

## Importar librerías

In [ ]:
!pip3 install flask_ngrok

### Importar el repositorio con las imágenes

In [ ]:
!git clone https://github.com/fcernafukuzaki/DogFaceNet-Dataset.git

Cloning into 'DogFaceNet-Dataset'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 9990 (delta 12), reused 58 (delta 9), pack-reused 9929
Receiving objects: 100% (9990/9990), 113.91 MiB | 47.13 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [ ]:
#!git pull https://github.com/fcernafukuzaki/DogFaceNet-Dataset.git

fatal: not a git repository (or any of the parent directories): .git


### Declarar constantes

In [ ]:
PATH        = './DogFaceNet-Dataset/data/dogfacenet/aligned/after_4_bis/'

#MODELO_ENTRENADO = '/content/drive/MyDrive/GHT/DogFaceNet_shared/DogFaceNet_output/output/model/2021.03.21_for-epochs10_fit-epochs20/2021.03.21_for-epochs10_fit-epochs20.dogfacenet.8.h5'
MODELO_ENTRENADO = '/content/drive/MyDrive/GHT/DogFaceNet_shared/DogFaceNet_output/output/modelo_tensorflow_entrenado/model/tensorflow.dogfacenet_tf.65.h5'
#MODELO_ENTRENADO = '/content/drive/MyDrive/Colab Notebooks/DogFaceNet/output/model2/2020.03.22.dogfacenet.9.h5'

REPOSITORIO_DE_IMAGENES_PUBLICAS = 'https://raw.githubusercontent.com/fcernafukuzaki/DogFaceNet-Dataset/main/data/dogfacenet/aligned/after_4_bis/'

PATH_2 = './static/'

### Ubicar archivos en Google Colab

### Subir archivos para despliegue en Google Colab

In [ ]:
import shutil

Para el despliegue de la aplicación en Flask en Google Colab se debe de subir 4 archivos:

- eval.py
- offline_training.py
- online_training.py
- index.html (Este archivo debe ir dentro de una carpeta "templates")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def cambiar_ubicacion_repositorio():
    original = './DogFaceNet-Dataset/data/dogfacenet/aligned/after_4_bis/'
    target = './static/'
    shutil.copytree(original,target)

    original_deploy = './DogFaceNet-Dataset/deploy/templates'
    target_deploy = './templates'
    shutil.copytree(original_deploy,target_deploy)

    original_deploy = './DogFaceNet-Dataset/deploy/eval.py'
    target_deploy = '/content/'
    shutil.copy(original_deploy,target_deploy)

    original_deploy = './DogFaceNet-Dataset/deploy/offline_training.py'
    target_deploy = '/content/'
    shutil.copy(original_deploy,target_deploy)

    original_deploy = './DogFaceNet-Dataset/deploy/online_training.py'
    target_deploy = '/content/'
    shutil.copy(original_deploy,target_deploy)

cambiar_ubicacion_repositorio()

### Importar librerías para el despliegue con Flask

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import tensorflow.keras.backend as K
import os
import numpy as np
import skimage as sk
import skimage.io
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from glob import glob
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename
import json
from flask_ngrok import run_with_ngrok
import itertools
import datetime

from online_training import *

## Modelo - Custom


In [ ]:
alpha = 0.3
def triplet(y_true,y_pred):
    
    a = y_pred[0::3]
    p = y_pred[1::3]
    n = y_pred[2::3]
    
    ap = K.sum(K.square(a-p),-1)
    an = K.sum(K.square(a-n),-1)

    return K.sum(tf.nn.relu(ap - an + alpha))

def triplet_acc(y_true,y_pred):
    a = y_pred[0::3]
    p = y_pred[1::3]
    n = y_pred[2::3]
    
    ap = K.sum(K.square(a-p),-1)
    an = K.sum(K.square(a-n),-1)
    
    return K.less(ap+alpha,an)

## Cargar Modelo

In [ ]:
def cargar_modelo(ruta_y_nombre_modelo):
  print('Loading model {}'.format(ruta_y_nombre_modelo))

  model = tf.keras.models.load_model(
      ruta_y_nombre_modelo,
      custom_objects={'triplet':triplet,'triplet_acc':triplet_acc})

  print('Done.')
  return model

model_1 = cargar_modelo(MODELO_ENTRENADO)

Loading model /content/drive/MyDrive/GHT/DogFaceNet_shared/DogFaceNet_output/output/modelo_tensorflow_entrenado/model/tensorflow.dogfacenet_tf.65.h5
Done.


In [ ]:
def predict_tensorflow(model, filenames_test):
    generator = predict_generator(filenames_test, 32) # filenames_test, batch=32
    steps=np.ceil(len(filenames_test)/32) # all
    predict=model.predict_generator(generator, steps=steps)
    return predict

In [ ]:
def obtener_ruta_de_imagen(directorio_imagenes):
    '''
    directorio_imagenes: Format /{numero}/{numero}.{identificador_foto}.jpg
    '''
    #assert os.path.isdir(directorio_imagenes), '[Error] Provided PATH for dataset does not exist.'

    print('Loading the dataset...')

    filenames = np.empty(0)
    labels = np.empty(0)
    idx = 0
    for root,dirs,files in os.walk(directorio_imagenes):
        #print('{} {} {}'.format(root,dirs,files))
        #if len(files)>1:
        if len(files)>0:
            for i in range(len(files)):
                files[i] = root + '/' + files[i]
            filenames = np.append(filenames,files)
            labels = np.append(labels,np.ones(len(files))*idx)
            idx += 1
    assert len(labels)!=0, '[Error] No data provided.'

    print('Done.')

    print('Total number of imported pictures: {:d}'.format(len(labels)))

    nbof_classes = len(np.unique(labels))
    print('Total number of classes: {:d}'.format(nbof_classes))
    return filenames, labels, nbof_classes

In [ ]:
def query_image_tensorflow(predict_imagen_cargada, predict, filenames_test, labels_test, distances):
    for i_dog_to_find in range(len(predict)):
        #print('\nPerro {}'.format(i_dog_to_find))
        emb1 = predict_imagen_cargada
        #print(emb1)

        differences = np.square(emb1 - predict)
        distances = np.sum(differences, 1)
        distances = list(distances)
        #print(distances)
        #print(len(distances))

        min_dist, max_dist = min(distances), max(distances)
        #print(min_dist, max_dist)

        images_and_distances = list(zip(itertools.count(), filenames_test, labels_test, distances))

        info_perro = images_and_distances[i_dog_to_find]
        #print("info_perro:", info_perro)

        imagenes_mismo_perro = [p for p in images_and_distances if p[2] == info_perro[2] and p[0] != info_perro[0]]
        #print("imagenes_mismo_perro:", imagenes_mismo_perro)

        #mismo_perro_indices, mismo_perro_filenames, mismo_perro_labels, mismo_perro_distances = zip(*imagenes_mismo_perro)
        #mostrar_imagenes(mismo_perro_filenames, [porcentaje(p) for p in mismo_perro_distances], "1_mismo_perro.png")

        # elimino la imagen buscada de entre los candidatos
        images_and_distances = [(i, filename, label, distance) for i, filename, label, distance in images_and_distances if i != i_dog_to_find]

        # los ordeno por similitud
        images_and_distances.sort(key=lambda x: x[3])

        # me quedo con los 12 mas parecidos
        #images_and_distances = images_and_distances[:16]

        #best_indices, best_filenames, best_labels, best_distances = zip(*images_and_distances)
        #formatear = lambda c, p: "SI! ### " + p + " ###" if c == info_perro[2] else p
        #best_texts = [formatear(p[2], porcentaje(p[3])) for p in images_and_distances]
        #mostrar_imagenes(best_filenames, best_texts, "2_encontrados.png")

        print('images_and_distances: {} '.format(images_and_distances))
        return images_and_distances

## Pruebas para mostrar TensorFlow

In [ ]:
def obtener_indices(list_rutas):
    seen = set()
    seen_add = seen.add
    return [x for x in list_rutas if not (x in seen or seen_add(x))]

In [ ]:
def obtener_todas_las_imagenes(PATH, nombre_carpeta):
    filenames = []
    for root, dirs, files in os.walk('{}{}'.format(PATH, nombre_carpeta)):
        if len(files)>0:
            for i in range(len(files)):
                files[i] = root.replace(PATH, "/static/") + '/' + files[i]
                filenames.append(files[i])
    return filenames

## Despliegue FLASK.

In [ ]:
app = Flask(__name__)

run_with_ngrok(app)

# DECLARAR VARIABLES
filenames_test, labels, nbof_classes = obtener_ruta_de_imagen(PATH_2)
predict_tensorflow_db_imagenes = predict_tensorflow(model_1, filenames_test)

@app.route('/')
@app.route('/index')
def index():
    return render_template('index_tf.html', title='Home',ims={})

@app.route('/search', methods=['GET','POST'])
def search_func():
    print('Inicio de búsqueda de imágen de mascota: {}'.format(datetime.datetime.now()))
    NOMBRE_IMAGEN_A_PREDECIR = "./image.jpg"
    clicked=None
    if request.method == "POST":
        file = request.files['img']
        file.save(NOMBRE_IMAGEN_A_PREDECIR)
    
    #static_img = "./static/imgs/"
    static_img = REPOSITORIO_DE_IMAGENES_PUBLICAS
    
    # Tensorflow
    print('Resultados de Tensorflow: {}'.format(datetime.datetime.now()))
    

    array_ruta_imagen_a_predecir = np.empty(0)
    array_ruta_imagen_a_predecir = np.append(array_ruta_imagen_a_predecir,NOMBRE_IMAGEN_A_PREDECIR)
    predict_tensorflow_imagen_cargada = predict_tensorflow(model_1, array_ruta_imagen_a_predecir)
    images_and_distances = query_image_tensorflow(predict_tensorflow_imagen_cargada, predict_tensorflow_db_imagenes, filenames_test, labels, nbof_classes)
    print('images_and_distances: {}'.format(images_and_distances))
    
    id_images_and_distances = obtener_indices([ruta.split('/')[-2] for _, ruta, _, _ in images_and_distances])[0:4]
    print(id_images_and_distances)

    results={}
    for i, img in enumerate(id_images_and_distances):
        #images_tf = f'{static_img}{img}/{img}.0.jpg'
        #results[f'results_tf{i+1}'] = images_tf
        results[f'results_tf{i+1}'] = obtener_todas_las_imagenes(PATH_2, img)
    #print('results_tf: {}'.format(results))
    
    print('Fin resultados Tensorflow: {}'.format(datetime.datetime.now()))

    print('results: {}'.format(results))
    print('Fin de búsqueda de imágen de mascota: {}'.format(datetime.datetime.now()))
    return results

Loading the dataset...
Done.
Total number of imported pictures: 8379
Total number of classes: 1396


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a4a77e675005.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [05/Apr/2021 15:38:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2021 15:38:13] "GET /favicon.ico HTTP/1.1" 404 -


Inicio de búsqueda de imágen de mascota: 2021-04-05 15:38:18.513189
Resultados de Tensorflow: 2021-04-05 15:38:18.662026


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
127.0.0.1 - - [05/Apr/2021 15:38:18] "POST /search HTTP/1.1" 200 -


images_and_distances: [(165, './static/444/444.2.jpg', 29.0, 0.00392487), (7636, './static/1023/1023.4.jpg', 1274.0, 0.011485536), (1179, './static/346/346.3.jpg', 188.0, 0.015895149), (7633, './static/1023/1023.2.jpg', 1274.0, 0.02440317), (7637, './static/1023/1023.0.jpg', 1274.0, 0.024709642), (28, './static/488/488.8.jpg', 5.0, 0.025477957), (896, './static/821/821.5.jpg', 145.0, 0.025498483), (32, './static/488/488.6.jpg', 5.0, 0.027522348), (423, './static/192/192.19.jpg', 68.0, 0.027589183), (34, './static/488/488.2.jpg', 5.0, 0.027911957), (29, './static/488/488.5.jpg', 5.0, 0.031044519), (430, './static/192/192.10.jpg', 68.0, 0.031303223), (169, './static/444/444.17.jpg', 29.0, 0.031915557), (5248, './static/203/203.13.jpg', 863.0, 0.035097495), (894, './static/821/821.0.jpg', 145.0, 0.036513433), (502, './static/1284/1284.4.jpg', 76.0, 0.038366098), (5218, './static/877/877.0.jpg', 859.0, 0.040526334), (6368, './static/1189/1189.3.jpg', 1046.0, 0.04131704), (26, './static/488

127.0.0.1 - - [05/Apr/2021 15:38:18] "GET /static/444/444.0.jpg HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2021 15:38:19] "GET /static/444/444.13.jpg HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2021 15:38:19] "GET /static/444/444.10.jpg HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2021 15:38:19] "GET /static/444/444.4.jpg HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2021 15:38:19] "GET /static/444/444.16.jpg HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2021 15:38:19] "GET /static/444/444.6.jpg HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2021 15:38:19] "GET /static/444/444.5.jpg HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2021 15:38:19] "GET /static/444/444.8.jpg HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2021 15:38:19] "GET /static/444/444.12.jpg HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2021 15:38:19] "GET /static/444/444.1.jpg HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2021 15:38:19] "GET /static/444/444.7.jpg HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2021 15:38:19] "GET /static/444/444.20.jpg HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2021 15:38:19